In [2]:
class Test:
    def __init__(self):
        print(self)
        return 1
    
test = Test()    

TypeError: __init__() should return None, not 'int'

In [51]:
class Test:
    def __init__(self, name):
        self.name = name
        
    def setmethod(self, value):
        self.value = value
        
test =  Test("Sam")
Test.setmethod(test, 3)

print(type(test.setmethod))
print(type(Test.setmethod))

print(Test.setmethod(test, 3) == test.setmethod(3))

<class 'method'>
<class 'function'>
True


In [52]:
class Person:
    def __init__(self, name):
        self.name = name
        print(self.name)
        
    def sayhello(self):
        print("Hi", self.name)
        
    def __del__(self):
        print("bye", self.name)
        
A = Person("Mayank")

A.sayhello()        
        

Mayank
Hi Mayank


In [53]:
del A


bye Mayank


In [56]:
class A:
    i = 123
    def __init__(self):
        self.i = 1234
        
    def f(self):
        print("Hi")
        
print(A.i)
print(A().i)
print(A().f());

123
1234
Hi
None


As is clear from above example, `__init__` function always return `None`

In [3]:
class Test:
    def __init__(self):
        print(self)
        print('x')

test = Test()        

x


`__init__` function is executed as soon as an instance of a class created.

In [4]:
class Test:
    def __init__(self):
        print("Test id is %d" %id(self)) 
        
x = Test()
y = Test()*10

Test id is 51907888
Test id is 51908016


TypeError: unsupported operand type(s) for *: 'Test' and 'int'

This program shows that object is instantiated before it is assigned to any variable. Line 1 runs fine. Line 2 produces `TypeError` because of wrong multiplication. Here instance was created (as shown by `id()` function) but its product with 10 produced error and nothing was assigned to `y`. Error happens only after instance is created and before instance is assigned a name 'y'. If this program is run interactively, `print(x)` command will run successfully but `print(y)` command will raise `NameError`. 

In [5]:
print(x)
print(y)

NameError: name 'y' is not defined

In [6]:
class Strings:
    def __init__(self,a):
        self.a = a
        
    def __add__(self, other):
        return self.a + other.a
    
a = Strings('x')
b = Strings('y')

a+b

'xy'

In [7]:
class Strings:
    def __init__(self,x,y):
        self.x = x
        self.y = y
        
    def __add__(self, other):
        self.x = self.x + other.x
        self.y = self.y + other.y
        return self.x, self.y
    
a = Strings(3,4)
b = Strings(4,5)

a + b


(7, 9)

In [11]:
class Employee:
    'common base class for all employees'
    empCount = 0
    
    def __init__(self, name, salary):
        self.name = name
        self.salary = salary
        Employee.empCount += 1
        
    def displayCount(self):
        print("Total No of Employee: %d" %Employee.empCount)
        
    def displayEmployee(self):
        print("Name: ", self.name, "Salary: ", self.salary)
        
emp1 = Employee('Sam',200)
emp2 = Employee('Peter',300)

print(Employee.empCount)
emp1.displayCount()
emp2.displayCount()
emp1.displayEmployee()
emp2.displayEmployee()


2
Total No of Employee: 2
Total No of Employee: 2
Name:  Sam Salary:  200
Name:  Peter Salary:  300


In [2]:
class C:
    def setname(self, who):
        self.name = who
        
I = C()
I.setname('Mayank')
I.name

'Mayank'

In [4]:
class C:
    def __init__(self,who):
        self.name = who
        
I = C('Mayank')    
I.name

'Mayank'

Here `__init__` is used instead of `setname`. This method is automatically called, whenever an instance is created. No need for extra method calls.

In [5]:
class Test:
    def __init__(self,arg1,arg2):
            self.arg1 = arg1
            self.arg2 = arg2
    
    def printargs(self):
        print(self.arg1)
        print(self.arg2)
        
instance = Test('a','b')

instance.printargs()
    

a
b


In [6]:
Test.printargs(instance)

a
b


Notice that `instance.printargs()` is equivalent to `Test.printargs(instance)`

In [16]:
class Employee:
    
    empCount = 0
    
    def __init__(self):
        Employee.empCount += 1
        print(Employee.empCount)
        print(id(self))

    def __del__(self):
        Employee.empCount -= 1
        print(Employee.empCount)
        print(id(self))
        
emp1 = Employee()
emp2 = Employee()

1
53761840
2
53761968


In [17]:
del emp1

1
53761840


In [18]:
del emp2

0
53761968


In [19]:
Employee.empCount

0

#### Encapsulation, Private Attributes and Data Hiding

In [23]:
class Encapsulation(object):
    def __init__(self, a, b, c):
        self.public = a
        self._protected = b
        self.__private = c
        print(self.__private)
        
A = Encapsulation(1,2,3)        

3


In [26]:
print(A.public)
print(A._protected)
print(A.__private)

1
2


AttributeError: 'Encapsulation' object has no attribute '__private'

As is clear from above, private attributes can't be accessed from outside. It can only be accessed from inside of class.

In [27]:
class Foo:
    def __init__(self, name):
        self.__name = name
    def setname(self):
        return self.__name
    
bar = Foo('Mayank')

bar.setname()

'Mayank'

In [28]:
bar.__name

AttributeError: 'Foo' object has no attribute '__name'

In [29]:
class Foo:
    def __init__(self, name):
        self.name = name
    def setname(self):
        return self.name
    
bar = Foo('Mayank')

bar.setname()

'Mayank'

In [30]:
bar.name

'Mayank'

Compare above 2 example. In first example, `bar.__name` raised `AttributeError` while in second example, `bar.name` ran correctly. 

In [32]:
class D:
    value =  5
    
class E(D):
    def printf(self):
        print(self.value)
        
a = E()
a.printf()

5


In [33]:
class D:
    __value =  5
    
class E(D):
    def printf(self):
        print(self.__value)
        
a = E()
a.printf()

AttributeError: 'E' object has no attribute '_E__value'

This illustrates that private attributes can't be accessed by child class as well.

From Python Essential Reference book – 

By default, all attributes and methods of a class are “public.”This means that they are all accessible without any restrictions. It also implies that everything defined in a base class is inherited and accessible within a derived class.This behavior is often undesirable in object-oriented applications because it exposes the internal implementation of an object and can lead to namespace conflicts between objects defined in a derived class and those defined in a base class. To fix this problem, all names in a class that start with a double underscore, such as` __Foo`, are automatically mangled to form a new name of the form `_Classname__Foo`. This effectively provides a way for a class to have private attributes and methods because private names used in a derived class won’t collide with the same private names used in a base class. 

Here is an example -
```python
class A(object):
    def __init__(self):
        self.__X = 3       # Mangled to self._A__X
    def __spam(self):      # Mangled to _A__spam()
        pass
    def bar(self):
        self.__spam()      # Only calls A.__spam()

class B(A):
    def __init__(self):
        A.__init__(self)
        self.__X = 37      # Mangled to self._B__X
    def __spam(self):      # Mangled to _B__spam()
        pass
```

Although this scheme provides the illusion of data hiding, there’s no strict mechanism in place to actually prevent access to the “private” attributes of a class. In particular, if the name of the class and corresponding private attribute are known, they can be accessed using the mangled name.


In [39]:
class Employee:
    def __init__(self, name):
        self.name = name
        
    def grade(self):
        print("Diff grade for diff group")
        
        
class Top(Employee):
    def __init__(self, name, perk):
        #Employee.__init__(self, name) #code is running even without this line. Why?
        self.perk = perk
    
    def grade(self):    
            Employee.grade(self)
            print("Top grade")
        
class Middle(Employee):
    def __init__(self, name, salary):
        Employee.__init__(self, name)
        self.salary =  salary
        
    def grade(self):
        print("Middle Grade")
        

emp1 = Top('Sam', 100)
emp2 = Middle('Peter', 200)

for e in [emp1, emp2]:
    print(e.grade());
    

Diff grade for diff group
Top grade
None
Middle Grade
None


In [40]:
class Super:
    def method(self):
        print("In super method")
        
class Sub(Super):
    def method(self):
        print("Starting sub method")
        Super.method(self)
        print("Ending sub method")
        
        
x = Super()
y = Sub()

for e in [x,y]:
    print(e.method())

In super method
None
Starting sub method
In super method
Ending sub method
None


In [44]:
class Customer:
    def __init__(self, name, balance):
        self.name = name
        self.balance =balance
    
    def deposit(self, amount):
        self.amount = amount
        self.balance = self.balance + amount
        print(self.balance)
        
    def test(self):
        print(amount)
        print(self.amount)
        
        
customer1 = Customer("Mayank", 1000)
customer2 = Customer("Kajal", 500)

customer1.deposit(500)
customer2.deposit(500)
    
    

1500
1000


In [43]:
customer1.test()

NameError: name 'amount' is not defined

'amount' can only be accessed by 'deposit' method. Here self.balance is called instance variable and amount would be local variable. Instance variable can be accessed by all methods defined in a class while local variables can be used by only that method in which they are defined.

In [58]:
class Test():
    pass

class Student(Test):
    pass

print(Test.__mro__)
print(Student.__mro__)

(<class '__main__.Test'>, <class 'object'>)
(<class '__main__.Student'>, <class '__main__.Test'>, <class 'object'>)


In [59]:
issubclass(Test, object)

True

In [60]:
issubclass(Student, object)

True

In [62]:
def foo():
    pass

issubclass(foo.__class__, object)

True

#### Static Method

In [63]:
class Test:

    @staticmethod
    def testmethod():
        print('Hi')
    
    def __init__(self,number):
        self.number = number

    def setmethod(self):
        print(self.number)    

Test.testmethod()       

Hi


#### Class Method

In [64]:
class Times():
    factor = 1
    @classmethod
    def mul(cls,x):
        return cls.factor*x

class TwoTimes(Times):
    factor = 2

x = TwoTimes.mul(4)    

print(x)

8


#### Properties

In [66]:
import math

class Circle:
    def __init__(self,radius):
        self.radius = radius

    @property
    def area(self):
        return math.pi*self.radius**2

    @property
    def perimeter(self):
        return 2*math.pi*self.radius

c = Circle(4)
c.radius



4

In [67]:
c.radius = 5
c.radius

5

In [68]:
c.perimeter

31.41592653589793

In [69]:
c.area

78.53981633974483

In [70]:
c.perimeter =  5

AttributeError: can't set attribute

As we know, instances of a class has attributes and methods bound to them. To access attributes and methods, we use dot notation. To access an attribute, we write `object.attribute` while to apply a method, we write `object.method()`. However, unlike methods, we can always change the value of attribute as shown above (c.radius = 5). In above program, if we don’t use `@property` decorator, we would apply method like this: `c.perimeter()`. But by using `@property` decorator, we get rid of those parentheses used after a method. So, now we can write `c.perimeter` instead of `c.perimeter()`. However, as said earlier, we can’t change the value of `c.perimeter` which is obvious. This is an example of making programming interface as uniform as possible.(Uniform Access Principle)	

#### `__repr__`

In [72]:
class Test:
	def __init__(self,value):
		self.value = value
	def __repr__(self):
		return 'value for current instance= %d' % self.value

a = Test(5)
a

value for current instance= 5

#### `__call__`

In [73]:
class Foo:
    def __call__(self):
        print('Hi')
        
bar = Foo()
bar()


Hi
